# Quick start

Let's set up a simple emulator for scattering of neutrons at various energies on various isotopes, using the [Koning-Delaroche](https://www.sciencedirect.com/science/article/pii/S0375947402013210?casa_token=qS1v6U4xDQEAAAAA:NIi9D5LpP3f05AMwRnvbQ6or8hSvXoEIgKBV56KA4l9aObCOVDAndmuCeIH77iuzoXMOOlAMyw) global optical potential for the neutron-nucleus interaction.

In [9]:
import rose
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

In [10]:
# useful constants
AMU = 931.494102  # MeV/c^2, Particle Data Group
MASS_N = 1.008665 * AMU  # MeV/c^2 PDG
MASS_P = 1.007276 * AMU  # MeV/c^2 PDG

In [11]:
def setup_system(A: int, Z: int,  energy_lab: float):
    """
    calculates the reduced mass, and the COM frame kinetic energy
    and wavenumber for a neutron scattering on a target nuclide
    Parameters:
        A : mass number of target
        Z : proton number of target
        energy_lab: bombarding energy in the lab frame [MeV]
    """
    N = A - Z

    # semi-empirical mass formula
    delta = 0
    if N%2 == 0 and Z%2 == 0:
        delta = 12.0 / np.sqrt(A)
    elif N%2 != 0 and Z%2 != 0:
        delta = - 12.0 / np.sqrt(A)

    Eb = (
        15.8 * A
      - 18.3 * A**(2/3)
      - 0.714 * Z*(Z-1)/(A**(1/3))
      - 23.2 * (N-Z)**2/A
      + delta
    )

    target_mass = Z * MASS_P + N * MASS_N - Eb
    mu = target_mass * MASS_N / (target_mass + MASS_N)
    energy_com = target_mass / (MASS_N + target_mass) * energy_lab

    return mu, energy_com, k

In [19]:
def get_kd_params(A, Z, Ecom):
    """
    Calculates Koning-Delaroche global neutron-nucleus OMP parameters for given A, Z; returns KD params
    and bounds for training determined by default parameters scaled by a factor of +/- x%
    """
    N = A - Z
    delta = (N - Z) / A

    v1 = 59.30 - 21.0 * delta - 0.024 * A
    v2 = 0.007228 - 1.48e-6 * A
    v3 = 1.994e-5 - 2.0e-8 * A
    v4 = 7.0e-9

    w1 = 12.195 + 0.0167 * A
    w2 = 73.55 + 0.0795 * A

    d1 = 16.0 - 16.0 * delta
    d2 = 0.0180 + 0.003802 / (1 + np.exp((A - 156.0) / 8.0))
    d3 = 11.5

    vso1 = 5.922 + 0.0030 * A
    vso2 = 0.0040

    wso1 = -3.1
    wso2 = 160

    Ef = 11.2814 + 0.02646 * A
    vv = rose.koning_delaroche.Vv(Ecom, v1, v2, v3, v4, Ef)
    rv = 1.3039 - 0.4054 * A ** (-1 / 3)
    av = 0.6778 - 1.487e-4 * A

    wv = rose.koning_delaroche.Wv(Ecom, w1, w2, Ef)
    rwv = rv
    awv = av

    wd = rose.koning_delaroche.Wd(Ecom, d1, d2, d3, Ef)
    rd = 1.3424 - 0.01585 * A ** (1 / 3)
    ad = 0.5446 - 1.656e-4 * A

    vso = rose.koning_delaroche.Vso(Ecom, vso1, vso2, Ef)
    wso = rose.koning_delaroche.Wso(Ecom, wso1, wso2, Ef)
    rso = 1.1854 - 0.647 * A ** (-1 / 3)
    aso = 0.59
    rwso = rso
    awso = aso

    R_C = 1.329

    # 15 params total
    params = np.array(
        [
            vv,
            rv * A ** (1 / 3),
            av,
            wv,
            rwv * A ** (1 / 3),
            awv,
            wd,
            rd * A ** (1 / 3),
            ad,
            vso,
            rso * A ** (1 / 3),
            aso,
            wso,
            rwso * A ** (1 / 3),
            awso,
        ]
    )
    return params

In [31]:
p =  get_kd_params(27,13, 14.1)

array([-5.36058763e-04,  2.90920000e+00,  5.90000000e-01])

In [23]:
# instantiate the interaction
interactions = rose.InteractionEIMSpace(
    rose.koning_delaroche.KD_simple,
    len(p),
    mu,
    Ecom,
    l_max,
    is_complex=True,
    spin_orbit_potential=rose.koning_delaroche.KD_simple_so,
    training_info=bounds,
    
)

In [24]:
from scipy.stats import qmc

# sample training and test points from within bounds
# 80/20% train/test division
test = qmc.scale(
    qmc.LatinHypercube(d=parameters.size).random(20), bounds[:, 0], bounds[:, 1]
)
train = qmc.scale(
    qmc.LatinHypercube(d=parameters.size).random(80), bounds[:, 0], bounds[:, 1]
)

In [25]:
# set up and train emulator using sensible defaults
emulator = rose.ScatteringAmplitudeEmulator.from_train(
    interactions,
    train,
    l_max,
    n_basis=20,
    angles=np.linspace(np.pi/181, np.pi, 180),
)

100%|███████████████████████████████████████████| 21/21 [02:11<00:00,  6.28s/it]


Now we have a trained emulator! Let's compare observables between the emulator and the high-fidelity solver using our test set of sampled parameters.

In [ ]:
%%time

# run high-fidelity solver
HIFI_xs = []
for sample in tqdm(test):
    HIFI_xs.append( emulator.exact_xs(sample))

 75%|████████████████████████████████▎          | 15/20 [04:28<01:30, 18.10s/it]

In [ ]:
%%time

# run emulator
emu_xs = []
for sample in tqdm(test):
    emu_xs.append( emulator.emulate_xs(sample))

Now let's plot the results!

In [ ]:
from matplotlib.lines import Line2D

fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
fig.patch.set_facecolor("white")

for i, sample in enumerate(test):
    x = emulator.angles * 180 / (np.pi)
    
    # HIFI
    y_exact = HIFI_xs[i].dsdo * emulator.rutherford
    
    # emulated
    y_emu = emu_xs[i].dsdo * emulator.rutherford

    p = ax.plot(x, y_emu, "--", alpha=0.5)
    ax.plot(x, y_exact, color=p[0].get_color(), alpha=0.5)

legend_styles = [
    Line2D([0], [0], color="k", linestyle="--", alpha=0.5),
    Line2D([0], [0], color="k", alpha=0.5),
]
ax.legend(legend_styles, ["emulated", "high-fidelity"])
ax.set_yscale("log")
plt.xlim([0, 180])
plt.xlabel(r"$\theta$ [$^\circ$]", fontsize=18)
plt.ylabel(r"$\frac{d \sigma}{d\Omega}$ [mb]", fontsize=16)

#plt.ylabel(r"$\sigma / \sigma_{\rm{Rutherford}}$ ", fontsize=16)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
fig.patch.set_facecolor("white")

for i, sample in enumerate(test):
    x = emulator.angles * 180/(np.pi)
    
    # HIFI
    y_exact = HIFI_xs[i].Ay 
    
    # emulated
    y_emu = emu_xs[i].Ay

    p = ax.plot(x, y_emu, "--", alpha=0.5)
    ax.plot(x, y_exact, color=p[0].get_color(), alpha=0.5)

legend_styles = [
    Line2D([0], [0], color="k", linestyle="--", alpha=0.5),
    Line2D([0], [0], color="k", alpha=0.5),
]
ax.legend(legend_styles, ["emulated", "high-fidelity"])
#ax.set_yscale("log")
plt.xlim([0, 180])
plt.xlabel(r"$\theta$ [$^\circ$]", fontsize=18)
plt.ylabel(r"$A_y(\theta)$ ", fontsize=16)
plt.show()